In [5]:
from typing import Dict, Callable, Any
import os

class RandomBlur:
    """Случайное размытие изображения"""
    def __init__(self, p=0.5, kernel_size=5):
        self.p = p
        self.kernel_size = kernel_size
    
    def __call__(self, img):
        if random.random() < self.p:
            # Конвертируем тензор в numpy
            img_np = img.numpy().transpose(1, 2, 0)
            # Применяем размытие
            blurred = cv2.GaussianBlur(img_np, (self.kernel_size, self.kernel_size), 0)
            return torch.from_numpy(blurred.transpose(2, 0, 1))
        return img
    
    
class RandomPerspective:
    """Случайная перспективная трансформация"""
    def __init__(self, p=0.5, distortion_scale=0.5):
        self.p = p
        self.distortion_scale = distortion_scale
    
    def __call__(self, img):
        if random.random() < self.p:
            # Конвертируем тензор в PIL
            to_pil = transforms.ToPILImage()
            img_pil = to_pil(img)
            
            # Параметры перспективы
            w, h = img_pil.size
            half_w = w // 2
            half_h = h // 2
            
            # Генерируем случайные смещения
            dx = random.randint(0, int(self.distortion_scale * half_w))
            dy = random.randint(0, int(self.distortion_scale * half_h))
            
            # Точки для перспективной трансформации
            src_points = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
            dst_points = np.float32([
                [dx, dy], 
                [w - dx, dy], 
                [0, h], 
                [w, h]
            ])
            
            # Применяем перспективную трансформацию
            M = cv2.getPerspectiveTransform(src_points, dst_points)
            img_np = np.array(img_pil)
            transformed = cv2.warpPerspective(img_np, M, (w, h))
            
            return transforms.ToTensor()(transformed)
        return img
    
class AugmentationPipeline:
    """Управляемый пайплайн для применения последовательности аугментаций"""
    
    def __init__(self):
        self.augmentations = {}
        self.order = []
    
    def add_augmentation(self, name: str, augmentation: Callable[[Any], Any], probability: float = 1.0):
        """
        Добавляет аугментацию в пайплайн
        
        Args:
            name: Уникальное имя аугментации
            augmentation: Функция/класс аугментации
            probability: Вероятность применения
        """
        if name in self.augmentations:
            raise ValueError(f"Аугментация с именем '{name}' уже существует")
        
        self.augmentations[name] = {
            'func': augmentation,
            'prob': probability
        }
        self.order.append(name)
    
    def remove_augmentation(self, name: str):
        """Удаляет аугментацию из пайплайна"""
        if name not in self.augmentations:
            raise KeyError(f"Аугментация '{name}' не найдена")
        
        del self.augmentations[name]
        self.order.remove(name)
    
    def apply(self, image):
        """Применяет последовательность аугментаций к изображению"""
        result = image
        
        for name in self.order:
            aug = self.augmentations[name]
            if random.random() < aug['prob']:
                result = aug['func'](result)
        
        return result
    
    def get_augmentations(self) -> Dict[str, dict]:
        """Возвращает информацию о всех аугментациях в пайплайне"""
        return {
            name: {
                'func': str(self.augmentations[name]['func']),
                'probability': self.augmentations[name]['prob']
            }
            for name in self.order
        }
    
    def __call__(self, image):
        """Позволяет использовать экземпляр как функцию"""
        return self.apply(image)
    
    
    
import torch
import torchvision.transforms as transforms
from augmentations_basics.extra_augs import *

def create_light_pipeline():
    """Легкие аугментации - минимальные изменения"""
    pipeline = AugmentationPipeline()
    
    pipeline.add_augmentation(
        "RandomHorizontalFlip",
        transforms.RandomHorizontalFlip(p=0.3),
        probability=0.5
    )
    
    pipeline.add_augmentation(
        "ColorJitter",
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        probability=0.5
    )
    
    return pipeline

def create_medium_pipeline():
    """Средние аугментации - умеренные изменения"""
    pipeline = AugmentationPipeline()
    
    pipeline.add_augmentation(
        "RandomHorizontalFlip",
        transforms.RandomHorizontalFlip(p=0.5),
        probability=0.7
    )
    
    pipeline.add_augmentation(
        "RandomRotation",
        transforms.RandomRotation(15),
        probability=0.7
    )
    
    pipeline.add_augmentation(
        "ColorJitter",
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        probability=0.7
    )
    
    pipeline.add_augmentation(
        "RandomBlur",
        RandomBlur(p=1.0, kernel_size=5),
        probability=0.3
    )
    
    return pipeline

def create_heavy_pipeline():
    """Тяжелые аугментации - значительные изменения"""
    pipeline = AugmentationPipeline()
    
    pipeline.add_augmentation(
        "RandomHorizontalFlip",
        transforms.RandomHorizontalFlip(p=0.7),
        probability=0.9
    )
    
    pipeline.add_augmentation(
        "RandomRotation",
        transforms.RandomRotation(30),
        probability=0.8
    )
    
    pipeline.add_augmentation(
        "ColorJitter",
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2),
        probability=0.8
    )
    
    pipeline.add_augmentation(
        "RandomPerspective",
        RandomPerspective(p=1.0, distortion_scale=0.4),
        probability=0.6
    )
    
    pipeline.add_augmentation(
        "RandomBlur",
        RandomBlur(p=1.0, kernel_size=9),
        probability=0.5
    )
    
    pipeline.add_augmentation(
        "CutOut",
        CutOut(p=1.0, size=(32, 32)),
        probability=0.4
    )
    
    pipeline.add_augmentation(
        "GaussianNoise",
        AddGaussianNoise(mean=0., std=0.15),
        probability=0.5
    )
    
    return pipeline

def print_pipeline_info(pipeline, name):
    """Печатает информацию о пайплайне"""
    print(f"\n {name} Pipeline")
    augs = pipeline.get_augmentations()
    for aug_name, info in augs.items():
        print(f"{aug_name}:")
        print(f"  Function: {info['func']}")
        print(f"  Probability: {info['probability']}")
    print()
    
    
from torchvision.transforms import ToTensor
from augmentations_basics.datasets import CustomImageDataset
import shutil
from tqdm import tqdm

def apply_and_save_pipeline(pipeline, dataset, output_dir, samples_per_class=5):
    """
    Применяет пайплайн к датасету и сохраняет результаты
    
    Args:
        pipeline: Экземпляр AugmentationPipeline
        dataset: Исходный датасет
        output_dir: Папка для сохранения результатов
        samples_per_class: Количество аугментированных образцов на класс
    """
    # Создаем структуру папок
    for class_name in dataset.classes:
        class_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
    
    # Преобразователь для PIL изображений
    to_tensor = ToTensor()
    
    # Собираем изображения для обработки
    images_to_process = []
    class_counts = {class_name: 0 for class_name in dataset.classes}
    
    for idx in range(len(dataset)):
        img, label = dataset[idx]
        class_name = dataset.classes[label]
        
        if class_counts[class_name] < samples_per_class:
            # Конвертируем PIL в тензор
            img_tensor = to_tensor(img)
            
            images_to_process.append({
                'original': img,
                'tensor': img_tensor,
                'class_name': class_name,
                'original_path': dataset.images[idx]
            })
            
            class_counts[class_name] += 1
        
        # Проверяем, собрали ли достаточно образцов
        if all(count >= samples_per_class for count in class_counts.values()):
            break
    
    # Применяем аугментации и сохраняем результаты
    for item in tqdm(images_to_process, desc=f"Processing {output_dir}"):
        class_name = item['class_name']
        original_name = os.path.splitext(os.path.basename(item['original_path']))[0]
        
        # Сохраняем оригинал
        original_path = os.path.join(output_dir, class_name, f"{original_name}_original.jpg")
        item['original'].save(original_path)
        
        # Применяем пайплайн 5 раз
        for i in range(5):
            # Применяем аугментации
            augmented = pipeline(item['tensor'].clone())
            
            # Конвертируем обратно в PIL
            augmented_pil = transforms.ToPILImage()(augmented)
            
            # Сохраняем результат
            save_path = os.path.join(
                output_dir, 
                class_name, 
                f"{original_name}_aug_{i+1}.jpg"
            )
            augmented_pil.save(save_path)

def main():
    # Пути к данным
    data_path = "data/train"
    output_base = "augmented_results"
    
    # Создаем датасет
    dataset = CustomImageDataset(
        root_dir=data_path,
        transform=None,
        target_size=(224, 224)
    )
    
    # Создаем конфигурации пайплайнов
    pipelines = {
        "light": create_light_pipeline(),
        "medium": create_medium_pipeline(),
        "heavy": create_heavy_pipeline()
    }
    
    # Печатаем информацию о пайплайнах
    for name, pipeline in pipelines.items():
        print_pipeline_info(pipeline, name)
    
    # Применяем и сохраняем результаты
    for config_name, pipeline in pipelines.items():
        output_dir = os.path.join(output_base, config_name)
        
        # Очищаем предыдущие результаты
        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)
        
        os.makedirs(output_dir, exist_ok=True)
        
        # Применяем пайплайн и сохраняем результаты
        apply_and_save_pipeline(
            pipeline=pipeline,
            dataset=dataset,
            output_dir=output_dir,
            samples_per_class=5
        )



main()


 light Pipeline
RandomHorizontalFlip:
  Function: RandomHorizontalFlip(p=0.3)
  Probability: 0.5
ColorJitter:
  Function: ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.1, 0.1))
  Probability: 0.5


 medium Pipeline
RandomHorizontalFlip:
  Function: RandomHorizontalFlip(p=0.5)
  Probability: 0.7
RandomRotation:
  Function: RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
  Probability: 0.7
ColorJitter:
  Function: ColorJitter(brightness=(0.7, 1.3), contrast=(0.7, 1.3), saturation=(0.7, 1.3), hue=(-0.1, 0.1))
  Probability: 0.7
RandomBlur:
  Function: <__main__.RandomBlur object at 0x0000017C7FA1DCD0>
  Probability: 0.3


 heavy Pipeline
RandomHorizontalFlip:
  Function: RandomHorizontalFlip(p=0.7)
  Probability: 0.9
RandomRotation:
  Function: RandomRotation(degrees=[-30.0, 30.0], interpolation=nearest, expand=False, fill=0)
  Probability: 0.8
ColorJitter:
  Function: ColorJitter(brightness=(0.5, 1.5), contrast=

Processing augmented_results\heavy: 100%|██████████| 30/30 [00:01<00:00, 16.68it/s]
